In [1]:
path <- '/home/User1/data/self_citations/model/'

### Predict papers n years

In [2]:
year_start = 5
diff_year = 10
year_pred = year_start+diff_year

In [3]:
total_years_file <- 20
col_status <- rep("NULL",total_years_file+1)
col_status[c(1,year_start+1,year_start+diff_year+1)] <- "integer"
data <- read.csv(file=paste0(path,'nb_articles.csv'),colClasses =col_status)

In [4]:
col_status <- rep("NULL",total_years_file+1)
col_status[c(1,year_start+1,year_start+diff_year+1)] <- "double"
types_cit <- c('self','co','others')
dfs_nb_cit <- list()
for(i in 1:length(types_cit)){
    type_cit <- types_cit[i]
    dfs_nb_cit[[i]] <- read.csv(file=paste0(path,'nb_citations_',type_cit,'.csv'),colClasses=col_status)
}

In [5]:
authors_info <- read.csv(file=paste0(path,'authors_infos_model.csv'),sep='\t')

In [6]:
for(i in 1:length(types_cit)){
    data <- merge(data,dfs_nb_cit[[i]],by="Cluster_ID")
}

In [7]:
rm(dfs_nb_cit)

In [8]:
data <- merge(data,authors_info,by="Cluster_ID")

In [9]:
nb_coauthors <- read.csv(file=paste0(path,'nb_coauthors.csv'),colClasses =col_status)

In [10]:
colnames(nb_coauthors)[c(2,3)] <- c(paste0("nb_co_",year_start),paste0("nb_co_",year_start+diff_year))

In [11]:
head(nb_coauthors)

,Cluster_ID,nb_co_5,nb_co_15
,<dbl>,<dbl>,<dbl>
1,31905805,5,5
2,10925507,7,7
3,37274983,9,33
4,11452931,13,13
5,38788724,5,5
6,43195937,3,3


In [12]:
col_status_char <- rep("NULL",total_years_file+1)
col_status_char[c(1,year_start+1,year_start+diff_year+1)] <- "character"

In [13]:
countries <- read.csv(file=paste0(path,'country_authors.csv'),colClasses =col_status_char)

In [14]:
data <- merge(data,nb_coauthors,by="Cluster_ID")

In [15]:
data <- merge(data,countries,by="Cluster_ID")

In [16]:
data$discipline <- as.character(data$discipline)

In [17]:
groups_disc <- read.csv(file=paste0(path,'groups_disc.csv'),header=F)

In [18]:
colnames(groups_disc) <- c("group","discipline")

In [19]:
data <- merge(data,groups_disc,by='discipline')

In [20]:
gender <- read.csv(file=paste0(path,'cluster_gender.txt'),sep='\t')

In [21]:
gender$gender <- tolower(gender$gender)

In [22]:
gender <- gender[gender$gender %in% c("m","f"),]

gender[gender$gender == "m","gender"] <- 1
gender[gender$gender == "f","gender"] <- 0
gender$gender <- as.numeric(gender$gender)

In [23]:
colnames(gender) <- c("Cluster_ID","gender")

In [24]:
data <- merge(data,gender,by='Cluster_ID')

In [37]:
head(data)

,Cluster_ID,discipline,nb_art_5,nb_art_15,nb_cits_self_5,nb_cits_self_15,nb_cits_co_5,nb_cits_co_15,nb_cits_others_5,nb_cits_others_15,min_year,max_year,nb_co_5,nb_co_15,country_5,country_15,group,gender,norm_co_5
,<int>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>,<dbl>,<chr>,<chr>,<fct>,<chr>,<dbl>
1,3,Chemistry,1,1,0.00,0.00,2.19,2.19,17.31,17.31,2013,2013,4,4,Sweden,Sweden,Natural Sciences and Engineering,m,4.000000
2,4,Chemistry,1,1,0.00,0.00,1.08,1.08,0.00,0.00,2015,2015,6,6,Sweden,Sweden,Natural Sciences and Engineering,f,6.000000
3,6,Mathematics,2,2,1.04,1.04,0.00,0.00,1.04,1.04,2016,2017,2,2,Tunisia,Tunisia,Natural Sciences and Engineering,m,1.000000
4,9,Physics,9,23,4.74,28.91,3.60,10.25,4.90,61.68,2009,2018,23,23,Italy,Italy,Natural Sciences and Engineering,m,2.555556
5,13,Physics,15,31,38.70,123.51,115.11,153.02,840.79,1130.13,2012,2018,78,275,Israel,Israel,Natural Sciences and Engineering,m,5.200000
6,28,Engineering and Technology,1,1,0.00,0.00,0.00,0.00,5.26,5.26,2016,2016,5,5,Russia,Russia,Natural Sciences and Engineering,m,5.000000


In [38]:
data[paste0("norm_cits_co_",year_start)] <- data[paste0("nb_cits_co_",year_start)]/data[paste0("nb_art_",year_start)]
data[paste0("norm_cits_self_",year_start)] <- data[paste0("nb_cits_self_",year_start)]/data[paste0("nb_art_",year_start)]
data[paste0("norm_cits_others_",year_start)] <- data[paste0("nb_cits_others_",year_start)]/data[paste0("nb_art_",year_start)]

In [39]:
# As time of start
min_year <- 1995
max_year <- 2007
# Min year car start
min_year_start  <- min_year - year_start
max_year_start <- max_year - year_start

data_model <- data[which((data['min_year'] >=min_year_start) & (data['min_year'] <= max_year_start)),]

In [40]:
list_disciplines <- unique(data_model$discipline)

In [41]:
act_from_start = year_pred
act_var <- paste0('active_',act_from_start)
data_model['nb_years_active'] <- data_model['max_year'] - data_model['min_year'] 
data_model[act_var] = data_model['nb_years_active'] >= act_from_start

In [42]:
data_model$group <- as.character(data_model$group)

In [43]:
col_status <- rep("NULL",total_years_file+1)
col_status[c(1,year_start+1,year_start+diff_year+1)] <- "numeric"
rank_authors <- read.csv(file=paste0(path,'rank_authors.csv'),colClasses =col_status)

In [44]:
data_model_rank <- merge(data_model,rank_authors,by="Cluster_ID")

In [45]:
groups <- unique(data_model$group)

#### Get stats

In [46]:
write.csv(data_model_rank,paste0(path,"data_authors_rank.csv"))

In [47]:
write.csv(data,paste0(path,"data_authors.csv"))